In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, Concatenate
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import cv2
import numpy as np
import os

def build_emotion_model(input_shape=(96, 96, 1), num_classes=7):
    """Build optimized emotion recognition model"""
    input_layer = Input(shape=input_shape)
    
    # Convert grayscale to 3 channels
    x = Concatenate()([input_layer, input_layer, input_layer])
    
    base_model = MobileNetV2(
        input_shape=(*input_shape[:2], 3),
        include_top=False,
        weights='imagenet',
        alpha=0.75
    )
    
    # Freeze initial layers, fine-tune later ones
    for layer in base_model.layers[:120]:
        layer.trainable = False
    for layer in base_model.layers[120:]:
        layer.trainable = True
        if isinstance(layer, tf.keras.layers.Conv2D):
            layer.kernel_regularizer = l2(0.0001)
    
    x = base_model(x)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.6)(x)
    x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    return Model(inputs=input_layer, outputs=outputs)

def create_data_generators(train_path, val_path, img_size=(96, 96), batch_size=64):
    """Create optimized data generators with augmentation"""
    if not os.path.exists(train_path):
        raise FileNotFoundError(f"Training directory not found: {train_path}")
    if not os.path.exists(val_path):
        raise FileNotFoundError(f"Validation directory not found: {val_path}")

    train_datagen = ImageDataGenerator(
        rotation_range=25,
        zoom_range=0.2,
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0.15,
        horizontal_flip=True,
        brightness_range=[0.85, 1.15],
        rescale=1./255,
        fill_mode='reflect',
        channel_shift_range=20.0
    )

    val_datagen = ImageDataGenerator(rescale=1./255)

    train_gen = train_datagen.flow_from_directory(
        train_path,
        target_size=img_size,
        color_mode="grayscale",
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True,
        seed=42
    )

    val_gen = val_datagen.flow_from_directory(
        val_path,
        target_size=img_size,
        color_mode="grayscale",
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=False
    )

    if len(train_gen.class_indices) == 0:
        raise ValueError("No training images found. Check your directory structure.")
    if len(val_gen.class_indices) == 0:
        raise ValueError("No validation images found. Check your directory structure.")

    return train_gen, val_gen

def train_model(model, train_gen, val_gen, save_path, epochs=100):
    """Train model with callbacks"""
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    callbacks = [
        ModelCheckpoint(
            save_path,
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
        ),
        EarlyStopping(
            monitor='val_accuracy',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1
        )
    ]

    model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        train_gen,
        epochs=epochs,
        validation_data=val_gen,
        callbacks=callbacks
    )

    return history
def detect_face(image, target_size=(96, 96)):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)

    if len(faces) == 0:
        return None

    (x, y, w, h) = faces[0]  # Take the first face
    face = gray[y:y+h, x:x+w]
    face_resized = cv2.resize(face, target_size)
    return face_resized


if __name__ == "__main__":
    # Configuration
    TRAIN_PATH = r"C:\Users\mahesh\Documents\facial_emotion_detection\images\train"
    VAL_PATH = r"C:\Users\mahesh\Documents\facial_emotion_detection\images\validation"
    MODEL_PATH = r"C:\Users\mahesh\Desktop\New folder\emotion_model_final.keras"
    IMG_SIZE = (96, 96)
    
    try:
        print("Building model...")
        model = build_emotion_model(input_shape=(*IMG_SIZE, 1))
        
        print("Creating data generators...")
        train_gen, val_gen = create_data_generators(TRAIN_PATH, VAL_PATH, IMG_SIZE)
        
        print("Starting training...")
        train_model(model, train_gen, val_gen, MODEL_PATH)
        
        print("Training complete. Starting detection...")
        detect_emotions(MODEL_PATH, IMG_SIZE)
    
    except Exception as e:
        print(f"Error occurred: {str(e)}")


In [15]:
import cv2
import numpy as np
import tensorflow as tf

# Load OpenCV's Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def detect_face(frame, img_size):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    if len(faces) == 0:
        return None
    x, y, w, h = faces[0]  # Take the first face detected
    face = gray[y:y+h, x:x+w]
    face = cv2.resize(face, img_size)
    return face


In [3]:
#for web cam facial expresion detection
def detect_emotions(MODEL_PATH, IMG_SIZE):
    model = tf.keras.models.load_model(MODEL_PATH)
    labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
    
    cap = cv2.VideoCapture(0)  # Live webcam feed
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        face_img = detect_face(frame, IMG_SIZE)
        if face_img is not None:
            norm_img = face_img.astype("float32") / 255.0
            norm_img = np.expand_dims(norm_img, axis=[0, -1])  # shape (1, 96, 96, 1)
            preds = model.predict(norm_img, verbose=0)[0]
            label = labels[np.argmax(preds)]
            confidence = np.max(preds)

            cv2.putText(frame, f"{label}: {confidence:.2f}", (10, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "No face detected", (10, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)

        cv2.imshow("Emotion Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Emotion labels (must match the model's training order)
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Load the trained emotion detection model
def load_emotion_model(model_path):
    return tf.keras.models.load_model(model_path)

# Detect all faces in an image using Haar Cascade
def detect_faces(image, img_size=(96, 96)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    face_list = []

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, img_size)
        face_list.append((face, (x, y, w, h)))

    return face_list

# Main function to detect emotion for multiple faces in one image
def detect_emotion_static(image_path, model_path, img_size=(96, 96)):
    model = load_emotion_model(model_path)

    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        print("❌ Image not found.")
        return

    # Detect all faces in the image
    faces = detect_faces(img, img_size)

    if not faces:
        print("❌ No faces detected.")
        return

    for face, (x, y, w, h) in faces:
        # Preprocess the face
        face_input = face.astype("float32") / 255.0
        face_input = np.expand_dims(face_input, axis=(0, -1))  # Shape: (1, 96, 96, 1)

        # Predict emotion
        preds = model.predict(face_input, verbose=0)[0]
        emotion = emotion_labels[np.argmax(preds)]
        confidence = np.max(preds)

        # Draw box and label on the image
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(img, f"{emotion} ({confidence:.2f})", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        print(f"✅ Detected Emotion: {emotion} ({confidence:.2f}) at [{x}, {y}, {w}, {h}]")

    # Show the final image with all detections
    cv2.imshow("Multiple Face Emotion Detection", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# ✅ Example usage
if __name__ == "__main__":
    model_path = r"C:\Users\mahesh\Desktop\New folder\emotion_model_final.keras"
    image_path = r"C:\Users\mahesh\Downloads\try2.webp" # Use image with multiple faces
    detect_emotion_static(image_path, model_path)


✅ Detected Emotion: neutral (0.66) at [181, 49, 57, 57]
✅ Detected Emotion: happy (0.62) at [582, 301, 57, 57]
✅ Detected Emotion: angry (0.93) at [585, 58, 63, 63]
✅ Detected Emotion: angry (0.84) at [184, 310, 60, 60]


In [17]:
MODEL_PATH = r"C:\Users\mahesh\Desktop\New folder\emotion_model_final.keras"

In [ ]:
# For real-time detection
detect_emotions(MODEL_PATH, (96, 96))